In [1]:
import os
import json
import re
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 1000)

In [3]:
data_path = 'dataset/train'
data = []
with open(data_path, 'r') as file:
    for line in file:
        data.append(json.loads(line.strip()))

data = pd.DataFrame(data)

# Разведочный анализ датасета

In [4]:
data.shape

(300000, 2)

In [5]:
data.head(300)

,dst,src
0,- Intriguing.,◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵
1,He would need to repeat his vows in the land of the living and drink from the wine of ages.,▽◪◎◗▦◫▦◫ ▫▴▨◓◠◓ ▴▫◎◪▱◫ ◚▴ ◞◧▦◞▾▢▱◨▨ ◒◠◓◠◀▪▦◈◠▦ ◫◉◎▴▱◫▵
2,You couldn't even answer my texts?,◄▴◞◠▸▱◠◓▪◎◠ ◀◫▱◪ ▼◪◚◠▻ ◚▴◓▴◎◪◈◗▦ ◎◫?
3,How fast do you go?,▯◪ ▨◠◈◠◓ ◞◭◓◠▫ ◳◠▻◬◳◧◓ ◞▴▦◗▦▨◫?
4,"He's talking about a few right here in Lisbon, mostly Jews who have escaped the Germans, that's all.",◈◠ ◧▱◠▦ ◀◫◓ ▨◠◉ ◂▱◠▽◈◠▦ ◀◠▷◞◪◈◗◳◧◓■ ◉◧◐▾▦▱◨◐▾ ○▱◎◠▦▱◠◓◈◠▦ ▨◠◉◠▦ ▽◠▷◨◈◫▱▴◓▵
5,"For instance, Ben is not a guy who skips work.",◝▴▦ ◫◒▫◪▦ ▨◠◳▫◠◓◠▼◠▨ ◀◗◓◫ ◈▴◐◫▱◈◗◓▵
6,"Goodbye, Jakob.",▭◂◒◉◠ ▨◠▱ ◡◠▨◂◀▵
7,Sure beats the foam domes round here.,◝◨◓◠◈◠▨◗ ▷◪◓▨◪◞◫ ◳◠▽◠ ◀◬◓◠▨▪◓▵
8,"Oh, wait a second!",◝◪▨▱▴ ◀◗◓ ◞◠▦◗▽▴!
9,The time is now.,◁◠◎◠▦ ◒◗◎◈◫▵


In [6]:
# Количество пропусков в каждом столбце
(data.isnull() | (data == '')).sum()

dst    0
src    0
dtype: int64

In [7]:
data.duplicated().sum()

26266

In [8]:
# Коэффициент кореляции между длинами предложений и их переводами
data_lens = data.map(lambda string : len(string))
data_lens.corr()

,dst,src
dst,1.00000,0.87585
src,0.87585,1.00000


In [9]:
# во сколько раз средняя длина английского текста больше средней длины текста ня языке Зетанов
data_lens['dst'].mean() / data_lens['src'].mean()

1.1232518826033402

In [10]:
# сделаем таблицу с выделенными из строк числами
EXTRACTING_EXPRESSION = r'\b\d{1,3}(?:[.,]\d{3})*(?:[.,]\d+)?\b'

extracted_nums = data.map(lambda string : re.findall(EXTRACTING_EXPRESSION, string))
extracted_nums[extracted_nums.any(axis=1)].head(20)

,dst,src
17,[12],[12]
54,"[00, 00, 46, 50, 13]",[]
73,[40],[40]
74,[50],[50]
91,[6],[6]
118,[127],[127]
135,[],[23]
160,[],[4]
165,[10],[10]
179,[45],[45]


In [11]:
# Колиество строк в которых числа различаются 
(extracted_nums['dst'] != extracted_nums['src']).sum()

4184

После разведочного анализа данных можно сделать следующие выходы:
- Похоже язык Зетанов посимвольный как и английский. Такое предположение можно сделать, заметив что почти в каждом тексте на этом языке в конце стоит символ "▵", скорее всего означающий точку.
- Разные предложение на английском могут иметь один перевод на язык Зетанов.
- Пропусков в датасете нет.
- В датасете есть дубликаты.
- Длинна предложения на английском имеет значительную кореляцию с длинной предложение на языке Зетанов.
- Длинны предложений на английском в среднем немного длиннее чем предложения на языке Зетанов.
- Цифры в языке Зетанов такие же как и в английском.

# Обработка данных

In [12]:
cleared_data = data.copy() 
# удаляем строки с несовпадающими числами
cleared_data = cleared_data[extracted_nums['dst'] == extracted_nums['src']]
# удаляем дубликаты
cleared_data = cleared_data.drop_duplicates() 
# удаляем строки где предложение и перевод слишком различаются по длинне
cleared_data = cleared_data[
    0.5 < (cleared_data['dst'].map(lambda string : len(string)) / cleared_data['src'].map(lambda string : len(string)) < 2.5)
]
cleared_data.reset_index(drop=True, inplace=True)

In [13]:
cleared_data.head(300)

,dst,src
0,- Intriguing.,◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵
1,He would need to repeat his vows in the land of the living and drink from the wine of ages.,▽◪◎◗▦◫▦◫ ▫▴▨◓◠◓ ▴▫◎◪▱◫ ◚▴ ◞◧▦◞▾▢▱◨▨ ◒◠◓◠◀▪▦◈◠▦ ◫◉◎▴▱◫▵
2,You couldn't even answer my texts?,◄▴◞◠▸▱◠◓▪◎◠ ◀◫▱◪ ▼◪◚◠▻ ◚▴◓▴◎◪◈◗▦ ◎◫?
3,How fast do you go?,▯◪ ▨◠◈◠◓ ◞◭◓◠▫ ◳◠▻◬◳◧◓ ◞▴▦◗▦▨◫?
4,"He's talking about a few right here in Lisbon, mostly Jews who have escaped the Germans, that's all.",◈◠ ◧▱◠▦ ◀◫◓ ▨◠◉ ◂▱◠▽◈◠▦ ◀◠▷◞◪◈◗◳◧◓■ ◉◧◐▾▦▱◨◐▾ ○▱◎◠▦▱◠◓◈◠▦ ▨◠◉◠▦ ▽◠▷◨◈◫▱▴◓▵
5,"For instance, Ben is not a guy who skips work.",◝▴▦ ◫◒▫◪▦ ▨◠◳▫◠◓◠▼◠▨ ◀◗◓◫ ◈▴◐◫▱◈◗◓▵
6,"Goodbye, Jakob.",▭◂◒◉◠ ▨◠▱ ◡◠▨◂◀▵
7,Sure beats the foam domes round here.,◝◨◓◠◈◠▨◗ ▷◪◓▨◪◞◫ ◳◠▽◠ ◀◬◓◠▨▪◓▵
8,"Oh, wait a second!",◝◪▨▱▴ ◀◗◓ ◞◠▦◗▽▴!
9,The time is now.,◁◠◎◠▦ ◒◗◎◈◫▵


In [14]:
cleared_data.shape

(266660, 2)

In [15]:
os.makedirs('cleared-dataset', exist_ok=True)
cleared_data.to_json('cleared-dataset/train', orient='records', force_ascii=False, lines=True)

## Краткий анализ валидационной и тестовой части

In [16]:
data_path = 'dataset/val'
data = []
with open(data_path, 'r') as file:
    for line in file:
        data.append(json.loads(line.strip()))

data = pd.DataFrame(data)

In [17]:
data['src'].duplicated().sum()

0

In [18]:
data_path = 'dataset/test_no_reference'
data = []
with open(data_path, 'r') as file:
    for line in file:
        data.append(json.loads(line.strip()))

data = pd.DataFrame(data)

In [19]:
data['src'].duplicated().sum()

0